In [1]:
%matplotlib notebook
import numpy as np
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import FlatLambdaCDM
import scipy.stats
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
plt.ion()

knn = 10
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
astorad = np.pi/(180.*60.*60)
Rauto_cut = 22.9
dz_membership = 0.02

# Load the classifications and photometry

In [2]:
classtable = Table().read('../catalogs/classified_objects.csv')
for i in range(knn):
    classtable[str(i+1)+'NN_arcsec'] = np.zeros(len(classtable))

phottable = Table().read('../catalogs/slits_phot_zs_cutonslitdist_rcl.csv')
phottable = phottable[np.where(phottable['Rauto'] < Rauto_cut)]
phottable = phottable[np.where(phottable['Q'] >= 3)]

# Find the k-NN (arcsec) and calculate the harmonic mean

In [3]:
for row in classtable:
    # Find nearby objects in z
    subtable = phottable[np.where(abs(phottable['zLDP'] - row['zLDP']) < dz_membership)]
    subcoords = SkyCoord(ra=subtable['ra']*u.degree, dec=subtable['dec']*u.degree)

    obj = SkyCoord(ra=row['ra']*u.degree, dec=row['dec']*u.degree)
    seps = subcoords.separation(obj).arcsecond
    kseps = sorted(seps)[1:knn+1]
    for i in range(knn):
        row[str(i+1)+'NN_arcsec'] = kseps[i]

# Calculate density in galaxies/Mpc$^2$

In [4]:
classtable['harm_mean'] = np.zeros(len(classtable))
classtable['surface_density'] = np.zeros(len(classtable))

for row in classtable:
    DA = cosmo.angular_diameter_distance(row['zLDP'])
    kseps = np.array([row[str(i+1)+'NN_arcsec'] for i in range(knn)]) * astorad * DA.value
    row['harm_mean'] = scipy.stats.hmean(kseps)
    
    # Account for the object itself!
    n_in_area = len(np.where(kseps<row['harm_mean'])[0]) + 1
    row['surface_density'] = n_in_area/(np.pi*row['harm_mean']*row['harm_mean'])

# Make the completeness correction
At the moment, this is to just divide by Q2 completeness at the relevent radius. Q2 since we are calculating the NN with Q=4,3,2 objects

In [5]:
completeness = Table().read('../catalogs/radial_completeness.dat', format='ascii')
completeness['theta_cl_min'] = completeness['theta_cl'] - 0.25
completeness['theta_cl_max'] = completeness['theta_cl'] + 0.25

In [6]:
classtable['surface_density_compcorred'] = np.zeros(len(classtable))
for row in classtable:
    rcl = (row['theta_cl_radian']*u.radian).to(u.arcminute).value
    for crow in completeness:
        if crow['theta_cl_min'] < rcl < crow['theta_cl_max']:
            row['surface_density_compcorred'] = row['surface_density']/crow['q3']

# Save the updated table

In [7]:
classtable.write('../catalogs/classified_surfacedensity.csv', format='csv', overwrite=True)

In [8]:
classtable

field,ids,ra,dec,x,y,B1,V1,R1,I1,z1,B2,V2,R2,I2,z2,B3,V3,R3,I3,z3,Bauto,Vauto,Rauto,Iauto,zauto,zLDP5_7,Q5_7,zphot,zphot_errUP,zphot_errLO,class_StarR,zLDP,zLDPerr,Q,zLDP_good,slit_distance,zSpec,zSpec_Q,spec_type,theta_cl_radian,d_cl_mpc,morph,morph_flags,1NN_arcsec,2NN_arcsec,3NN_arcsec,4NN_arcsec,5NN_arcsec,6NN_arcsec,7NN_arcsec,8NN_arcsec,9NN_arcsec,10NN_arcsec,harm_mean,surface_density,surface_density_compcorred
str9,str17,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,str3,str14,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
CL1018,WFI_1018-12_11513,154.67478942871094,-12.298959732055664,5167.25732421875,3294.1455078125,23.630199432373047,22.55335235595703,21.630298614501953,20.978435516357422,-2.5,22.6025333404541,21.749500274658203,20.92743682861328,20.213401794433594,-2.5,22.341535568237305,21.56652069091797,20.7730712890625,20.04343032836914,-2.5,22.314748764038086,21.54250717163086,20.75238037109375,20.028173446655273,-2.5,0.4659999907016754,4,0.5870000123977661,0.03700000047683716,0.03700000047683716,0.02800000086426735,0.467707097530365,2.473452241247287e-06,4.0,1.0,0.42897423845538124,-1.0,-1,--,0.0017954807053920975,2.1944365181302214,Sc,--,66.52569007221798,93.99493223534307,119.72746924185618,127.46953005347008,129.45613031453595,140.83817572571317,155.12372321397987,159.15097744977314,160.17716537109789,162.18808002445272,0.7192910102879382,2.4609390269189664,8.381057136746564
CL1054-12,WFI_1054-12_6840,163.71055603027344,-12.952329635620117,3525.17041015625,1929.3612060546875,24.13758087158203,23.260997772216797,22.46585464477539,21.58448600769043,21.258249282836914,23.660598754882812,22.882598876953125,22.030895233154297,21.081777572631836,20.716434478759766,23.545059204101562,22.937313079833984,21.98180389404297,21.00802993774414,20.61372184753418,23.615928649902344,22.882474899291992,22.002756118774414,21.051000595092773,20.66661262512207,0.7670000195503235,4,0.6980000138282776,0.06599999964237213,0.06400000303983688,0.3630000054836273,0.7655920386314392,6.79316417517839e-06,4.0,1.0,0.4124658811424027,-1.0,-1,--,0.0033226922665082754,5.028230206906973,C,--,23.483351032986604,25.9859157520251,78.53047750194064,117.3342392775592,124.12278305882408,135.8530058824775,137.56094162266226,151.28587318989435,168.5396186339109,169.87705844347212,0.5154418877358983,3.594280479584544,14.728112688175539
cl1059,WFI_1059-12_9876,164.81338500976562,-12.931870460510254,3573.239013671875,4109.87109375,25.674983978271484,23.477462768554688,22.535654067993164,21.67604637145996,-2.5,24.46158218383789,22.90027618408203,22.02737045288086,21.20659828186035,-2.5,24.099252700805664,22.76567268371582,21.93603515625,21.139842987060547,-2.5,24.14120864868164,22.818256378173828,21.958114624023438,21.151145935058594,-2.5,0.4580000042915344,4,0.6830000281333923,0.03099999949336052,0.029999999329447746,0.6480000019073486,0.4639773666858673,3.365164502611151e-06,4.0,1.0,0.10907402473976655,-1.0,-1,--,0.0009533303557120142,1.141613100965137,E,--,15.980091372941398,31.758989723541394,32.53154012137539,36.94361483050298,44.48030457265146,54.882878226459,60.934251456375144,68.47081905453776,70.16428374238049,70.86584039704283,0.2325744762919713,29.423614929024396,103.31956387291008
cl1059,WFI_1059-12_9492,164.81561279296875,-12.941900253295898,3540.517822265625,3958.14306640625,24.74534034729004,22.94040870666504,22.4869327545166,22.116493225097656,-2.5,23.5399227142334,22.571815490722656,22.158451080322266,21.888118743896484,-2.5,23.226547241210938,22.534591674804688,22.11349868774414,22.002946853637695,-2.5,23.473575592041016,22.558494567871094,22.150150299072266,21.90378761291504,-2.5,0.46299999952316284,4,0.298000007867